In [ ]:
!pip install pandas openpyxl

### Imports:

In [ ]:
import pandas as pd
import json
import os

### Upload Excel file



In [ ]:
from google.colab import files
uploaded = files.upload()

Saving com_urls.xlsx to com_urls.xlsx


### Set up environment (read the file, create output folder)


In [ ]:
excel_filename = list(uploaded.keys())[0]

In [ ]:
df = pd.read_excel(excel_filename, dtype={'Page ID': str})

In [ ]:
output_folder = "com_links"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
df.columns = df.columns.str.strip()
print("Column Names Detected:")
print(df.columns.tolist())

Column Names Detected:
['URL', 'Result Block Title', 'Result Block Author', 'Result Block Text', 'Details', 'Page ID']


### Save information from the excel file into separate json files for data processing later.

In [ ]:
for index, row in df.iterrows():
    page_id = str(row['Page ID']).strip()

    # Prepare dictionary: convert datetime and other non-serializable objects to strings
    json_data = {
        "url": str(row['URL']),
        "page_id": str(row['Page ID']),
        "result_block_title": str(row['Result Block Title']),
        "result_block_author": str(row['Result Block Author']),
        "result_block_text": str(row['Result Block Text']),
        "details": str(row['Details'])
    }

    # Save JSON
    json_filename = os.path.join(output_folder, f"{page_id}.json")
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)

print(f"✅ All JSON files saved in '{output_folder}'.")

✅ All JSON files saved in 'com_links'.


### Add Page Rank Data

The jsons do not have the Page Rank data that I collected. This following section adds the page rank data from the search results. 0 means Featured Snippet

In [ ]:

uploaded = files.upload()
txt_file_name = list(uploaded.keys())[0]

print(f"Uploaded: {txt_file_name}")

id_to_ranks = {}



Saving page_rankings.txt to page_rankings.txt
Uploaded: page_rankings.txt


### Parse the Txt File

In [ ]:
with open(txt_file_name, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        # Each line format: "<WEBPAGE_ID> <RANK> <SEARCH_TERM>"
        parts = line.split(" ", 2)
        if len(parts) < 3:
            continue  # skip malformed lines
        webpage_id, rank_value, search_term = parts[0], parts[1], parts[2]
        entry = {
            "search_query": search_term,
            "rank": rank_value
        }
        if webpage_id not in id_to_ranks:
            id_to_ranks[webpage_id] = []
        id_to_ranks[webpage_id].append(entry)


### Add data to jsons

In [ ]:
json_folder = "./com_links"

for webpage_id, rank_entries in id_to_ranks.items():
    json_path = os.path.join(json_folder, f"{webpage_id}.json")

    if not os.path.exists(json_path):
        print(f"Warning: JSON file {json_path} does not exist, skipping.")
        continue

    with open(json_path, "r", encoding="utf-8") as jf:
        try:
            data = json.load(jf)
        except json.JSONDecodeError:
            print(f"Warning: Could not decode JSON in {json_path}, skipping.")
            continue

    # Add or update "result_rank"
    if "result_rank" not in data:
        data["result_rank"] = []
    data["result_rank"].extend(rank_entries)

    with open(json_path, "w", encoding="utf-8") as jf:
        json.dump(data, jf, ensure_ascii=False, indent=2)

print("✅ All matching JSON files updated!")

✅ All matching JSON files updated!


# Part 2: Scraping

In [ ]:
!pip install beautifulsoup4 requests chardet

In [ ]:
import os
import json
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import chardet
import time

### Setup

In [ ]:
input_folder = 'com_links'

In [ ]:
output_folder = 'com_links_scraped'
os.makedirs(output_folder, exist_ok=True)

### Data collection section

In [ ]:
print(f"Starting scraping... Total files: {len(os.listdir(input_folder))}")

for idx, filename in enumerate(os.listdir(input_folder), 1):
    if filename.endswith('.json'):
        input_filepath = os.path.join(input_folder, filename)
        output_filepath = os.path.join(output_folder, filename)

        with open(input_filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)

        url = data.get('URL') or data.get('url')
        if not url:
            print(f"[{idx}] {filename}: No URL found, skipping.")
            continue

        # --- Skip PDFs ---
        if url.lower().endswith('.pdf'):
            print(f"[{idx}] {filename}: URL is a PDF, skipping.")
            continue

        print(f"\n[{idx}] Processing: {filename}")
        print(f"    URL: {url}")

        try:
            # --- Track timing ---
            start_time = time.time()

            # GET request
            response = requests.get(url, timeout=20, headers={'User-Agent': 'Mozilla/5.0'})
            download_time = time.time() - start_time

            # --- Encoding detection ---
            detected = chardet.detect(response.content)
            encoding = detected['encoding'] if detected['confidence'] > 0.5 else 'utf-8'
            print(f"    Downloaded in {download_time:.2f}s | Encoding: {encoding}")

            # --- Parsing ---
            soup = BeautifulSoup(response.content, 'html.parser', from_encoding=encoding)

            # --- Extract fields ---
            title = soup.title.string.strip() if soup.title else None

            pub_date = None
            pub_date_tag = soup.find('meta', property='article:published_time') or soup.find('meta', attrs={'name': 'pubdate'})
            if pub_date_tag and pub_date_tag.get('content'):
                pub_date = pub_date_tag['content'].strip()

            # Main text
            paragraphs = [p.get_text(strip=True) for p in soup.find_all('p')]
            headings = [h.get_text(strip=True) for h in soup.find_all(['h1', 'h2', 'h3'])]
            main_text = "\n".join(headings + paragraphs)

            # Embedded media
            embedded_media = []
            for iframe in soup.find_all('iframe', src=True):
                embedded_media.append(iframe['src'])
            for video in soup.find_all('video', src=True):
                embedded_media.append(video['src'])

            # Add website_data
            data['website_data'] = {
                'webpage_title': title,
                'publication_date': pub_date,
                'main_text': main_text[:10000],
                'embedded_media': embedded_media
            }

            # Save updated JSON
            with open(output_filepath, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4)

            print(f"    ✅ Completed and saved to '{output_folder}/{filename}'")

        except Exception as e:
            print(f"    ❌ Failed to fetch {url}: {e}")

print(f"\n✅ Scraping complete! JSONs saved in '{output_folder}'.")


Starting scraping... Total files: 90

[1] Processing: 9968944052.json
    URL: https://zhidao.baidu.com/question/51172329.html
    Downloaded in 2.60s | Encoding: utf-8
    ✅ Completed and saved to 'com_links_scraped/9968944052.json'

[2] Processing: 1814451065.json
    URL: http://www.baidu.com/from=844b/ssid=0/s?word=%E5%8D%81%E5%A4%A7%E5%BF%AB%E9%80%9F%E6%AD%BB%E4%BA%A1%E6%97%A0%E7%97%9B%E6%96%B9%E6%B3%95&sa=re_dl_prs_34689_5&ms=1&rqid=11657973654154055419&rq=%E7%9D%A1%E8%A7%89%E6%AD%BB%E4%BA%A1%E6%B3%95&rsf=1630013&asctag=58000
    Downloaded in 2.30s | Encoding: utf-8
    ✅ Completed and saved to 'com_links_scraped/1814451065.json'

[3] Processing: 1885847504.json
    URL: http://www.wellyclinic.tw/h/ServiceDetail?key=3501106059&cont=2503&set=7
    Downloaded in 0.61s | Encoding: utf-8
    ✅ Completed and saved to 'com_links_scraped/1885847504.json'

[4] Processing: 5899489299.json
    URL: https://zhuanlan.zhihu.com/p/539664390
    Downloaded in 0.64s | Encoding: utf-8
    ✅ Comp

    Downloaded in 2.21s | Encoding: utf-8
    ✅ Completed and saved to 'com_links_scraped/9702304018.json'

[37] Processing: 4969809966.json
    URL: https://sbhk.org.hk/?page_id=34172
    Downloaded in 2.27s | Encoding: utf-8
    ✅ Completed and saved to 'com_links_scraped/4969809966.json'

[38] Processing: 9467082684.json
    URL: https://www.commonhealth.com.tw/article/82568
    Downloaded in 0.03s | Encoding: ascii
    ✅ Completed and saved to 'com_links_scraped/9467082684.json'

[39] Processing: 4903961193.json
    URL: https://www.mghclaycenter.org/parenting-concerns/%E9%A2%84%E9%98%B2%E8%87%AA%E6%9D%80%E5%A6%82%E4%BD%95%E5%B8%AE%E5%8A%A9%E6%82%A8%E7%9A%84%E5%AD%A9%E5%AD%90/
    Downloaded in 0.30s | Encoding: utf-8
    ✅ Completed and saved to 'com_links_scraped/4903961193.json'

[40] Processing: 0314968668.json
    URL: https://m.iask.sina.com.cn/b/1SSTprcUntw7.html
    Downloaded in 1.34s | Encoding: utf-8
    ✅ Completed and saved to 'com_links_scraped/0314968668.json'

[41] 

### Cleanup -- lots of websites not detected by Beautifulsoup

In [ ]:
# Paths to your folders
com_links_folder = 'com_links'
scraped_folder = 'com_links_scraped'

# Get list of filenames in each folder
com_links_files = set(os.listdir(com_links_folder))
scraped_files = set(os.listdir(scraped_folder))

# Find files in com_links not in scraped
missing_files = com_links_files - scraped_files

# Output
print(f"Total files in '{com_links_folder}': {len(com_links_files)}")
print(f"Total files in '{scraped_folder}': {len(scraped_files)}")
print(f"\nFiles not yet scraped ({len(missing_files)}):")

for filename in sorted(missing_files):
    print(filename)

Total files in 'com_links': 90
Total files in 'com_links_scraped': 82

Files not yet scraped (8):
0234222609.json
1249805490.json
2074393071.json
5330054929.json
6680502017.json
7422813570.json
7695045207.json
8197407423.json


### Detect missing or misformatted data so that I can input it into the jsons directly.

In [ ]:
folder_path = 'com_links_scraped'

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            website_data = data.get('website_data', {})
            urll = data.get('url')
            title = website_data.get('webpage_title', 'N/A')
            main_text = website_data.get('main_text', '')
            truncated_text = main_text[:100] + ('...' if len(main_text) > 100 else '')

            print(f"Filename: {filename}")
            print(f"URL: {urll}")
            print(f"Title: {title}")
            print(f"Main Text (truncated): {truncated_text}")
            print("="*80)

        except Exception as e:
            print(f"Error processing {filename}: {e}")

Filename: 9968944052.json
URL: https://zhidao.baidu.com/question/51172329.html
Title: 百度安全验证
Main Text (truncated): 问题反馈
Filename: 1814451065.json
URL: http://www.baidu.com/from=844b/ssid=0/s?word=%E5%8D%81%E5%A4%A7%E5%BF%AB%E9%80%9F%E6%AD%BB%E4%BA%A1%E6%97%A0%E7%97%9B%E6%96%B9%E6%B3%95&sa=re_dl_prs_34689_5&ms=1&rqid=11657973654154055419&rq=%E7%9D%A1%E8%A7%89%E6%AD%BB%E4%BA%A1%E6%B3%95&rsf=1630013&asctag=58000
Title: 十大快速死亡无痛方法 - 百度
Main Text (truncated): 十大快速死亡无痛方法？ - 权威心理科专家解答 - 百度健康
推荐权威三甲专家
十大快速死亡无痛方法？ - 权威心理科专家解答 - 百度健康
刑侦知识-十种不会痛的死亡  掌握刑侦知识,了解十种无痛苦...
Filename: 1885847504.json
URL: http://www.wellyclinic.tw/h/ServiceDetail?key=3501106059&cont=2503&set=7
Title: 服用安眠藥會致癌、致死嗎？
Main Text (truncated): 服用安眠藥會致癌、致死嗎？
聯繫我們 / Contact Us
黃偉俐診所(代碼：2621)
APP 下載
今年一月份英國一篇醫學文章探討安眠藥使用者的死亡與罹癌率，跟不服用的同年齡對照組相比，即使一...
Filename: 5899489299.json
URL: https://zhuanlan.zhihu.com/p/539664390
Title: 安静死亡是一种乐 - 知乎
Main Text (truncated): 安静死亡是一种乐
如何安静的去死，连尸体都找不到的那种。且死前不会被阻止 不会反悔 不被陌生人拦截。
我只是想要安静的死去 没有烦心事 没有

### Delete bad files (i.e. link can't be opened, repetitive link)

In [ ]:
id_list_input = "6060248314, 0431867372, 1422741122"  # Replace this line!

# === Parse IDs ===
ids_to_delete = [id_str.strip() for id_str in id_list_input.split(',')]

# === Folder with JSON files ===
folder_path = 'com_links_scraped'

# === Track deletions ===
deleted_files = []
not_found_ids = []

# === Delete matching files ===
for id_str in ids_to_delete:
    found = False
    for filename in os.listdir(folder_path):
        if filename.endswith('.json') and id_str in filename:
            file_path = os.path.join(folder_path, filename)
            try:
                os.remove(file_path)
                deleted_files.append(filename)
                print(f"Deleted: {filename}")
                found = True
                break  # Assumes one file per ID. Remove this line if multiple files per ID are possible.
            except Exception as e:
                print(f"Error deleting {filename}: {e}")
                found = True
                break
    if not found:
        not_found_ids.append(id_str)

# === Summary ===
print("\nSummary:")
print(f"Deleted files: {deleted_files}")
if not_found_ids:
    print(f"No file found for IDs: {not_found_ids}")
else:
    print("All specified IDs had matching files.")

Deleted: 6060248314.json
Deleted: 0431867372.json
Deleted: 1422741122.json

Summary:
Deleted files: ['6060248314.json', '0431867372.json', '1422741122.json']
All specified IDs had matching files.


### Need to add manual scraping into the jsons

In [ ]:
import shutil

In [ ]:
com_links_folder = 'com_links'
scraped_folder = 'com_links_scraped'

In [ ]:
uploaded = files.upload()
excel_file2 = list(uploaded.keys())[0]
df = pd.read_excel(excel_file2)

Saving manual_scrape.xlsx to manual_scrape.xlsx


In [ ]:
print(f"Loaded Excel with {len(df)} rows")

# Process each row
for idx, row in df.iterrows():
    file_id = str(row['ID']).strip()
    page_title = str(row['Page Title']).strip() if pd.notnull(row['Page Title']) else None
    page_body = str(row['Page Body']).strip() if pd.notnull(row['Page Body']) else None

    filename = f"{file_id}.json"
    input_path = os.path.join(com_links_folder, filename)
    output_path = os.path.join(scraped_folder, filename)

    # Check if exists in com_links folder
    if os.path.exists(input_path):
        print(f"[{idx+1}] Processing {filename}")

        # Load JSON
        with open(input_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Add website_data
        data['website_data'] = {
            'webpage_title': page_title,
            'publication_date': None,
            'main_text': page_body[:10000] if page_body else '',
            'embedded_media': []
        }

        # Delete existing file in scraped folder if exists
        if os.path.exists(output_path):
            os.remove(output_path)
            print(f"    Existing file in scraped folder removed: {output_path}")

        # Save updated JSON to scraped folder
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)

        print(f"    ✅ Saved updated JSON to: {output_path}")

    else:
        print(f"[{idx+1}] File {filename} not found in '{com_links_folder}', skipping.")

print("\n✅ All matching files processed.")

Loaded Excel with 30 rows
[1] Processing 3081280474.json
    Existing file in scraped folder removed: com_links_scraped/3081280474.json
    ✅ Saved updated JSON to: com_links_scraped/3081280474.json
[2] Processing 1512327726.json
    Existing file in scraped folder removed: com_links_scraped/1512327726.json
    ✅ Saved updated JSON to: com_links_scraped/1512327726.json
[3] Processing 7695045207.json
    ✅ Saved updated JSON to: com_links_scraped/7695045207.json
[4] Processing 8812631163.json
    Existing file in scraped folder removed: com_links_scraped/8812631163.json
    ✅ Saved updated JSON to: com_links_scraped/8812631163.json
[5] Processing 5010859106.json
    Existing file in scraped folder removed: com_links_scraped/5010859106.json
    ✅ Saved updated JSON to: com_links_scraped/5010859106.json
[6] Processing 7422813570.json
    ✅ Saved updated JSON to: com_links_scraped/7422813570.json
[7] Processing 8197407423.json
    ✅ Saved updated JSON to: com_links_scraped/8197407423.json


### Download Cleaned Data

In [ ]:
folder_to_zip = 'com_links_scraped'
output_zip = 'com_links_scraped.zip'

# Create zip file
shutil.make_archive('com_links_scraped', 'zip', folder_to_zip)
print(f"✅ Folder '{folder_to_zip}' zipped as '{output_zip}'")

# Download zip file
files.download(output_zip)

✅ Folder 'com_links_scraped' zipped as 'com_links_scraped.zip'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>